# Dissolve the SHN file again so I can use the same `map tiles` as the other data products
* <i>Just need to ensure your shn gdf has Route,  County, District, RouteType columns.</i>

In [1]:
import geopandas as gpd
import google.auth
import numpy as np
import pandas as pd

credentials, project = google.auth.default()

import gcsfs

fs = gcsfs.GCSFileSystem()
import yaml

In [26]:
from calitp_data_analysis import geography_utils, utils
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import webmap_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [3]:
SHN_FILE = catalog_utils.get_catalog("shared_data_catalog").state_highway_network.urlpath

shn = gpd.read_parquet(
        SHN_FILE,
        storage_options={"token": credentials.token},
    ).to_crs(geography_utils.CA_NAD83Albers_ft)

In [4]:
shn.sample()

,Route,County,District,RouteType,Direction,geometry
971,280,SM,4,Interstate,SB,"MULTILINESTRING ((-714011.252 9017582.423, -71..."


### Dissolve it the same way but add back County/Route Type

In [5]:
shn_dissolved = (shn.dissolve(by=["Route", "District"]).reset_index())[["Route", "District", "geometry"]]

In [6]:
# Subset
shn_wo_geo = shn.drop(columns = ["geometry", "Direction"])

In [9]:
shn_wo_geo.sample()

,Route,County,District,RouteType
948,267,PLA,3,State


In [10]:
agg1 = (
        shn_wo_geo.groupby(
            ["Route", "District"],  # maybe comment out later
            as_index=False,
        )
        .agg(
            {
                "County": lambda x: ", ".join(set(x.astype(str))),
                "RouteType": lambda x: ", ".join(set(x.astype(str))),
            }
        )
        .reset_index(drop=True)
    )


In [13]:
agg1.sample(10)

,Route,District,County,RouteType
53,32,3,"BUT, GLE",State
20,9,4,"SCL, SM",State
94,59,6,MAD,State
32,15,11,SD,"State, Interstate"
310,269,6,"FRE, KIN",State
112,72,12,ORA,State
252,188,11,SD,State
97,60,8,"SBD, RIV",State
102,65,6,"TUL, KER",State
303,260,4,ALA,State


### Merge

In [14]:
m1 = pd.merge(shn_dissolved, agg1, on = ["Route", "District"], indicator = True)

In [15]:
m1._merge.value_counts()

both          344
left_only       0
right_only      0
Name: _merge, dtype: int64

In [16]:
type(m1)

geopandas.geodataframe.GeoDataFrame

## 2nd Approach, just buffer the original map each time.

In [56]:
def load_buffered_shn_map(buffer_amount:int, district:int) -> gpd.GeoDataFrame:
    """
    Load buffered and dissolved version of the SHN that we can
    use with the webmaps.
    """
    SHN_FILE = catalog_utils.get_catalog("shared_data_catalog").state_highway_network.urlpath

    gdf = gpd.read_parquet(
        SHN_FILE,
        storage_options={"token": credentials.token},
    ).to_crs(geography_utils.CA_NAD83Albers_ft)
    
    # Filter for the relevant district
    gdf2 = gdf.loc[gdf.district == district]
    
    # Dissolve
    gdf2 = gdf2.dissolve(by = ["Route","County","District", "RouteType"]).reset_index().drop(columns = ["Direction"])
    
    # Buffer
    gdf2.geometry = gdf2.geometry.buffer(buffer_amount)
    
    # Rename the columns
    gdf2 = gdf2.rename(columns = shn_map_readable_columns)
    
    # Change the CRS
    gdf2 = gdf2.to_crs(geography_utils.CA_NAD83Albers_m)
    return gdf2

In [43]:
shn.sample()

,Route,County,District,RouteType,Direction,geometry
525,99,SJ,10,State,SB,"LINESTRING (-320535.414 9020876.522, -321247.0..."


In [44]:
shn_d4 = shn.loc[shn.District == 4]

In [45]:
shn_d4.shape

(174, 6)

In [46]:
shn_d4 = shn_d4.dissolve(by = ["Route","County","District", "RouteType"]).reset_index().drop(columns = ["Direction"])

In [47]:
shn_d4.shape

(87, 5)

In [48]:
shn_d4.sample()

,Route,County,District,RouteType,geometry
21,37,SOL,4,State,"MULTILINESTRING ((-638736.951 9176394.057, -63..."


In [49]:
shn_d4.geometry = shn_d4.geometry.buffer(50)

In [50]:
shn_d4.sample()

,Route,County,District,RouteType,geometry
22,37,SON,4,State,"POLYGON ((-719797.932 9168969.782, -719799.443..."


In [53]:
shn_d4 = shn_d4.to_crs(geography_utils.CA_NAD83Albers_m)

In [51]:
# shn_d4.explore()

In [54]:
shn_map = webmap_utils.set_state_export(
    shn_d4,
    filename="test_cc_routes1",
    map_type = 'state_highway_network',
    map_title="SHN route",
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [55]:
webmap_utils.display_spa_map(shn_map["spa_link"])